In [ ]:
# THIS IS A SEPARATE FILE ISOLAING THE DERIVATION OF THE GAIN MATRIX AND DEFINITION OF CONTROLLER CLASS #
# THIS WAS DONE SUCH THAT THE MAIN FILE ONLY CONTAINS NECESSARY INFORMATION                             #
# ADDITIONALLY THIS ALLOWS FOR EASIER ANNOTATION OF THE CODE                                            #
#########################################################################################################
# THE FOLLOWING DERIVATION OF THE GAIN MATRIX IS TAKEN FROM:                                            #
# "Lecture Notes - MIT16_30F10_lec18.pdf"                                                               #
#########################################################################################################
import numpy as np
from scipy.linalg import solve_continuous_are

# Define the system matrices A and B
A = np.array([
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, -7.77, 0, 0],
    [0, 33.67, 0, 0]
])

B = np.array([
    [0],
    [0],
    [0.57],
    [-1.63]
])

# Define the state cost matrix
# The state cost matrix is a diagonal matrix of shape n x n where n is the number of states in the system
Q = np.eye(A.shape[0])
# Q is defined as a 4x4 identity matrix

# Define the control cost matrix
# The control cost matrix is a diagonal matrix of shape m x m where m is the number of control inputs in the system
R = np.eye(B.shape[1])
# R is defined as a 1x1 identity matrix

# Using scipy.linalg.solve_continuous_are function to solve the algebraic Riccati equation expressed in the form (A - B K) P + P A.T + Q = 0
# Where P is the solution to the Riccati equation and K is the subsequent gain matrix
P = solve_continuous_are(A, B, Q, R)
K = np.linalg.inv(R) @ B.T @ P

# The gain matrix for the controller was acting incorrectly, so through trial and error I multiplied it by a scaling factor
# With the understanding that the terms of the gain matrix affect the respective terms in the state-space relation
K = K * np.array([10,4,9,10])
# The aforementioned trial and error yielded the following gain matrix
K = np.array([-10, -186.24428252, -33.6012562, -89.09737686])
# Which is the final gain matrix used in the controller in the main file "Catbotproject.ipynb"

class Controller:
    def __init__(self, K):
        self.K = K
    
    def reset(self):
        pass
    
    def run(self, t, wheel_position, wheel_velocity, pitch_angle, pitch_rate, cat_target):
        # Define state vector
        x = np.array([[wheel_position], [pitch_angle], [wheel_velocity], [pitch_rate]])
        
        # Define the dynamic reference state 
        x_ref = np.array([[cat_target], [0], [0], [0]])
        
        # Compute control input 
        t = -self.K @ (x - x_ref)
        
        # Ensure torque is within limits (magnitude of 5 N*m)
        return np.clip(t.item(), -5, 5)